In [2]:
import pandas as pd
import string
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer

# Ensure necessary NLTK resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jacobemerson/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jacobemerson/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jacobemerson/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jacobemerson/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
jobs_df = pd.read_csv("/Users/jacobemerson/code/cgallin/cover_genie/raw_data/job_data/postings.csv")
jobs_df.head(5)

,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,...,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,normalized_salary,zip_code,fips
0,921716,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,20.0,HOURLY,"Princeton, NJ",2774458.0,20.0,NaN,...,Requirements: \n\nWe are seeking a College or ...,1.713398e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,38480.0,8540.0,34021.0
1,1829192,NaN,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ...",50.0,HOURLY,"Fort Collins, CO",NaN,1.0,NaN,...,NaN,1.712858e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,83200.0,80521.0,8069.0
2,10998357,The National Exemplar,Assitant Restaurant Manager,The National Exemplar is accepting application...,65000.0,YEARLY,"Cincinnati, OH",64896719.0,8.0,NaN,...,We are currently accepting resumes for FOH - A...,1.713278e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,55000.0,45202.0,39061.0
3,23221523,"Abrams Fensterman, LLP",Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...,175000.0,YEARLY,"New Hyde Park, NY",766262.0,16.0,NaN,...,This position requires a baseline understandin...,1.712896e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,157500.0,11040.0,36059.0
4,35982263,NaN,Service Technician,Looking for HVAC service tech with experience ...,80000.0,YEARLY,"Burlington, IA",NaN,3.0,NaN,...,NaN,1.713452e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,70000.0,52601.0,19057.0


In [4]:
# def preprocessor(df):
#     def text_cleaner(df):
#         # Ensure 'description' column exists
#         if 'description' not in df.columns:
#             raise ValueError("DataFrame must contain a 'description' column.")
        
#         # Drop rows with missing 'description' values
#         df_cleaned = df.dropna(subset=['description'])
        
#         # Convert text to lowercase and remove punctuation
#         translator = str.maketrans('', '', string.punctuation)
#         df_cleaned['description'] = df_cleaned['description'].str.lower().str.translate(translator)
        
#         # Remove numbers
#         df_cleaned['description'] = df_cleaned['description'].str.replace(r'\d+', '', regex=True)
        
#         return df_cleaned['description']

#     text = text_cleaner(df)

#     # Tokenize the text
#     def tokenizer(texts):
#         return texts.apply(word_tokenize)

#     tokens = tokenizer(text)

#     # Remove stopwords
#     def remove_stopwords(tokens):
#         stop_words = set(stopwords.words('english')).union(stopwords.words('french'))
#         return tokens.apply(lambda words: [w for w in words if w not in stop_words])

#     no_stopword_tokens = remove_stopwords(tokens)

#     # Lemmatize text
#     def lemmatize(tokens):
#         lemmatizer = WordNetLemmatizer()
        
#         def lemmatize_words(words):
#             # Lemmatize verbs and nouns
#             lemmatized_verbs = [lemmatizer.lemmatize(word, pos='v') for word in words]
#             lemmatized_nouns = [lemmatizer.lemmatize(word, pos='n') for word in lemmatized_verbs]
#             return lemmatized_nouns
        
#         return tokens.apply(lemmatize_words)

#     lemmatized_tokens = lemmatize(no_stopword_tokens)

#     # Return lemmatized tokens as a new DataFrame column
#     df['processed_description'] = lemmatized_tokens
#     return pd.DataFrame(df['processed_description'])
    


In [5]:
# def preprocessor(df):
#     # Clean and preprocess text
#     def text_cleaner(df):
#         if 'description' not in df.columns:
#             raise ValueError("DataFrame must contain a 'description' column.")
        
#         # Drop rows with missing descriptions
#         df_cleaned = df.dropna(subset=['description']).copy()
        
#         # Apply regex to clean text (keep only letters, replace others with space)
#         df_cleaned['description'] = df_cleaned['description'].apply(lambda x: re.sub('[^a-zA-Z]', ' ', x))
        
#         # Convert to lowercase
#         df_cleaned['description'] = df_cleaned['description'].str.lower()
        
#         return df_cleaned['description']

#     # Process sentences within the cleaned text
#     def process_sentences(texts):
#         # Set up stopwords and lemmatizer
#         stop_words = set(stopwords.words('english')).union(stopwords.words('french'))
#         lemmatizer = WordNetLemmatizer()
        
#         # Extract features from sentences
#         def extract_features(text):
#             features = {'feature': ""}
#             sentences = sent_tokenize(text)
#             for sent in sentences:
#                 # Tokenize, remove stopwords, and filter by POS tags
#                 words = word_tokenize(sent)
#                 words = [word for word in words if word not in stop_words]
#                 tagged_words = pos_tag(words)
#                 filtered_words = [word for word, tag in tagged_words if tag not in ['DT', 'IN', 'TO', 'PRP', 'WP']]
                
#                 # Lemmatize remaining words
#                 lemmatized_words = [lemmatizer.lemmatize(word, pos='v') for word in filtered_words]
#                 lemmatized_words = [lemmatizer.lemmatize(word, pos='n') for word in lemmatized_words]
                
#                 # Append to features
#                 features['feature'] += " ".join(lemmatized_words) + " "
#             return features['feature']
        
#         # Apply feature extraction to all texts
#         return texts.apply(extract_features)
    
#     # Apply text cleaning
#     cleaned_text = text_cleaner(df)
    
#     # Process sentences and extract features
#     processed_text = process_sentences(cleaned_text)
    
#     # Add processed text as a new column
#     df['processed_description'] = processed_text
#     return df[['processed_description']].copy()

In [6]:
def preprocessor(text):
    # Clean the text
    def text_cleaner(text):
        # Apply regex to clean text (keep only letters, replace others with space)
        cleaned_text = re.sub('[^a-zA-Z]', ' ', text)
        
        # Convert to lowercase
        cleaned_text = cleaned_text.lower()
        
        return cleaned_text

    # Process sentences within the cleaned text
    def process_sentences(text):
        # Set up stopwords and lemmatizer
        stop_words = set(stopwords.words('english')).union(stopwords.words('french'))
        lemmatizer = WordNetLemmatizer()
        
        # Extract features from sentences
        def extract_features(text):
            features = ""
            sentences = sent_tokenize(text)
            for sent in sentences:
                # Tokenize, remove stopwords, and filter by POS tags
                words = word_tokenize(sent)
                words = [word for word in words if word not in stop_words]
                tagged_words = pos_tag(words)
                filtered_words = [word for word, tag in tagged_words if tag not in ['DT', 'IN', 'TO', 'PRP', 'WP']]
                
                # Lemmatize remaining words
                lemmatized_words = [lemmatizer.lemmatize(word, pos='v') for word in filtered_words]
                lemmatized_words = [lemmatizer.lemmatize(word, pos='n') for word in lemmatized_words]
                
                # Append to features
                features += " ".join(lemmatized_words) + " "
            return features.strip()
        
        # Apply feature extraction to the text
        return extract_features(text)
    
    # Apply text cleaning
    cleaned_text = text_cleaner(text)
    
    # Process sentences and extract features
    processed_text = process_sentences(cleaned_text)
    
    return processed_text


In [7]:
# Apply the preprocessor function to the first 100 rows of the 'description' column
jobs_df.loc[:99, 'processed_description'] = jobs_df.loc[:99, 'description'].apply(preprocessor)


In [8]:
jobs_df.head()

,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,...,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,normalized_salary,zip_code,fips,processed_description
0,921716,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,20.0,HOURLY,"Princeton, NJ",2774458.0,20.0,NaN,...,1.713398e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,38480.0,8540.0,34021.0,job descriptiona lead real estate firm new jer...
1,1829192,NaN,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ...",50.0,HOURLY,"Fort Collins, CO",NaN,1.0,NaN,...,1.712858e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,83200.0,80521.0,8069.0,aspen therapy wellness commit serve client bes...
2,10998357,The National Exemplar,Assitant Restaurant Manager,The National Exemplar is accepting application...,65000.0,YEARLY,"Cincinnati, OH",64896719.0,8.0,NaN,...,1.713278e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,55000.0,45202.0,39061.0,national exemplar accept application assistant...
3,23221523,"Abrams Fensterman, LLP",Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...,175000.0,YEARLY,"New Hyde Park, NY",766262.0,16.0,NaN,...,1.712896e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,157500.0,11040.0,36059.0,senior associate attorney elder law trust esta...
4,35982263,NaN,Service Technician,Looking for HVAC service tech with experience ...,80000.0,YEARLY,"Burlington, IA",NaN,3.0,NaN,...,1.713452e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,70000.0,52601.0,19057.0,look hvac service tech experience commerical i...


In [9]:
resumes_df = pd.read_csv('/Users/jacobemerson/code/cgallin/cover_genie/raw_data/archive-3/Resume/Resume.csv')
resumes_df.head(5)

,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR


In [10]:
resumes_df = resumes_df.rename(columns={'Resume_str':'description'})

In [11]:
resumes_df.loc[:99, 'processed_description'] = resumes_df.loc[:99, 'description'].apply(preprocessor)

In [12]:
# # Combine all text data for fitting the vectorizer
# resumes = resumes_df.loc[:99, 'processed_description'].str.split().sum()
# job_desc = jobs_df.loc[:99, 'processed_description'].str.split().sum()

In [13]:
# # Vectorizing the text data
# vectorizer = TfidfVectorizer()
# job_desc_tfidf = vectorizer.fit_transform(resumes_df.loc[:99, 'processed_description'])
# resume_tfidf = vectorizer.transform(jobs_df.loc[:99, 'processed_description'])

In [14]:
# from sklearn.metrics.pairwise import cosine_similarity

# tfidf_similarity_matrix = cosine_similarity(job_desc_tfidf, resume_tfidf)

In [15]:
# # Define the number of top jobs to review
# k = 5

# for resume_idx in range(tfidf_similarity_matrix.shape[1]):  # Loop over resumes
#     top_jobs = tfidf_similarity_matrix[:, resume_idx].argsort()[::-1][:k]
#     print(f"Resume {resume_idx}: Top {k} jobs")
#     for job_idx in top_jobs:
#         similarity_score = tfidf_similarity_matrix[job_idx, resume_idx]
#         print(f"  Job {job_idx}, Similarity: {similarity_score:.4f}")

In [16]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity

# def recommendation(resume_texts, job_texts, k=5):
#     """
#     Recommends top k jobs for each resume based on text similarity.

#     Parameters:
#     - resume_texts: List or Series of resume descriptions (processed).
#     - job_texts: List or Series of job descriptions (processed).
#     - k: Number of top job recommendations for each resume.

#     Returns:
#     - None (prints recommendations for each resume).
#     """
#     # Ensure inputs are lists of strings
#     if isinstance(resume_texts, list):
#         resume_texts = pd.Series(resume_texts)
#     if isinstance(job_texts, list):
#         job_texts = pd.Series(job_texts)

#     # Vectorizing the text data
#     vectorizer = TfidfVectorizer()
#     job_desc_tfidf = vectorizer.fit_transform(job_texts)
#     resume_tfidf = vectorizer.transform(resume_texts)

#     # Compute the cosine similarity between job descriptions and resumes
#     tfidf_similarity_matrix = cosine_similarity(job_desc_tfidf, resume_tfidf)

#     # Iterate through each resume and find the top k most similar jobs
#     for resume_idx in range(tfidf_similarity_matrix.shape[1]):  # Loop over resumes
#         top_jobs = tfidf_similarity_matrix[:, resume_idx].argsort()[::-1][:k]
#         print(f"Resume {resume_idx}: Top {k} jobs")
#         for job_idx in top_jobs:
#             similarity_score = tfidf_similarity_matrix[job_idx, resume_idx]
#             print(f"  Job {job_idx}, Similarity: {similarity_score:.4f}")


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def recommendation(resume_text, job_texts, k=5):
    """
    Recommends top k jobs for a given resume based on text similarity.

    Parameters:
    - resume_text: A single resume description (processed).
    - job_texts: List or Series of job descriptions (processed).
    - k: Number of top job recommendations.

    Returns:
    - List of tuples containing job indices and similarity scores for the top k jobs.
    """
    # Ensure inputs are in the correct format
    if isinstance(job_texts, list):
        job_texts = pd.Series(job_texts)

    # Vectorizing the text data
    vectorizer = TfidfVectorizer()
    job_desc_tfidf = vectorizer.fit_transform(job_texts)
    resume_tfidf = vectorizer.transform([resume_text])  # Transform single resume

    # Compute the cosine similarity between the resume and all job descriptions
    tfidf_similarity_scores = cosine_similarity(job_desc_tfidf, resume_tfidf).flatten()

    # Find the top k most similar jobs
    top_jobs = tfidf_similarity_scores.argsort()[::-1][:k]
    top_job_scores = [(job_idx, tfidf_similarity_scores[job_idx]) for job_idx in top_jobs]

    return top_job_scores


In [18]:
# Extract the first 100 processed descriptions
resume_texts = resumes_df.loc[:99, 'processed_description']
job_texts = jobs_df.loc[:99, 'processed_description']

# Iterate through each resume and get recommendations
for idx, resume_text in enumerate(resume_texts):
    print(f"Recommendations for Resume {idx}:")
    top_jobs = recommendation(resume_text, job_texts, k=5)
    for job_idx, similarity in top_jobs:
        print(f"  Job {job_idx}, Similarity: {similarity:.4f}")

Recommendations for Resume 0:
  Job 99, Similarity: 0.2606
  Job 35, Similarity: 0.2411
  Job 95, Similarity: 0.2365
  Job 14, Similarity: 0.2278
  Job 30, Similarity: 0.2236
Recommendations for Resume 1:
  Job 99, Similarity: 0.3563
  Job 35, Similarity: 0.3056
  Job 51, Similarity: 0.2930
  Job 14, Similarity: 0.2845
  Job 0, Similarity: 0.2809
Recommendations for Resume 2:
  Job 58, Similarity: 0.2764
  Job 30, Similarity: 0.2666
  Job 16, Similarity: 0.2548
  Job 89, Similarity: 0.2470
  Job 84, Similarity: 0.2054
Recommendations for Resume 3:
  Job 95, Similarity: 0.2345
  Job 81, Similarity: 0.2204
  Job 16, Similarity: 0.1880
  Job 10, Similarity: 0.1824
  Job 30, Similarity: 0.1727
Recommendations for Resume 4:
  Job 58, Similarity: 0.3069
  Job 30, Similarity: 0.2988
  Job 16, Similarity: 0.2755
  Job 57, Similarity: 0.2162
  Job 94, Similarity: 0.1861
Recommendations for Resume 5:
  Job 58, Similarity: 0.2896
  Job 30, Similarity: 0.2568
  Job 92, Similarity: 0.2473
  Job 81,

In [40]:
recommendation(resume_text, job_texts, k=5)

[(16, 0.190907179122512),
 (89, 0.1509899563592097),
 (77, 0.1464134479782347),
 (10, 0.14254506135357167),
 (58, 0.14140504090377692)]

In [41]:
jobs_df.loc[16, ['description', 'title']]

description    Job Title: Administrative CoordinatorOrganizat...
title                                 Administrative Coordinator
Name: 16, dtype: object

In [42]:
print(jobs_df['description'][16])

Job Title: Administrative CoordinatorOrganization: Cleveland Kids’ Book BankLocation: MidTown Cleveland WarehousePosition Type: F/T 40 hours per weekHourly Rate: $25Schedule: Monday – Friday 9:00 am – 5:00 pmReports to: Executive DirectorSupervises: NoneDescription:The Cleveland Kids’ Book Bank seeks an Administrative Coordinator who will play a pivotal rolein ensuring the successful operation of administrative functions within the organization. Thismultifaceted position encompasses various responsibilities, including administrative fiscalmanagement, office management and purchasing, customer relationship management (CRM /donor database) and administrative functions for HR and employee benefits.Key Responsibilities:Fiscal Coordination:Process invoices, expense reports, and vendor payments accurately and in a timely manner.Monitor accounts receivable, follow up on outstanding payments, and maintain records of transactions.Update QuickBooks software to track financial transactions, gener

In [ ]:
print(resumes_df['description'][99
                                ])

         HR BENEFITS/LEAVE COORDINATOR       Summary    13 years of Human Resources experience and 27 years of administrative experience working in various settings
*Professional, detail-oriented, excellent time-management skills, team player, open and honest communicator, effective at maintaining confidentiality and multi-tasker.      Skills        Proficient in all Microsoft applications and HRIS systems (Banner, PeopleAdmin, Personnel Management Information (PMIS) System and Benefits Eligibility System (BES), VRS Navigator (VNAV))            Experience     December 2016   to   Current     Company Name   City  ,   State     HR Benefits/Leave Coordinator        Provide consultation and assistance for all employee benefits.  Assist Retirement/Benefits Manager in counseling employees on retirements as well as generate retirement calculations.  Assist Retirement/Benefits Manager with retirement trainings.  Promote and coordinate all aspects of the Highlander Wellness Watch program.  Advi

In [22]:
import seaborn as sns
import matplotlib.pyplot as plt

# Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(tfidf_similarity_matrix, cmap='viridis', annot=False)
plt.title("TF-IDF Cosine Similarity Heatmap (Jobs vs Resumes)")
plt.xlabel("Resumes")
plt.ylabel("Jobs")
plt.show()

NameError: name 'tfidf_similarity_matrix' is not defined

<Figure size 1000x800 with 0 Axes>